In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
%matplotlib inline

from sklearn.preprocessing import LabelEncoder

from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC


from sklearn.model_selection import GridSearchCV


from sklearn.metrics import roc_curve, roc_auc_score, classification_report, accuracy_score, confusion_matrix



In [2]:
training = pd.read_csv('training_set_features.csv')  # Training set features

In [3]:
for i, row in training[training['employment_occupation'].isna()].iterrows():
    conditions = (training["employment_status"]=="Unemployed")
    if conditions.any():
        training.loc[i, 'employment_occupation'] = 'unemployed_occupation'

In [4]:
for i, row in training[training['employment_occupation'].isna()].iterrows():
    conditions = (training["employment_status"]=="Not in Labor Force")
    if conditions.any():
        training_data.loc[i, 'employment_occupation'] = 'Not_in_laborforce'

In [5]:
for i, row in training[training['employment_occupation'].isna()].iterrows():
    conditions = (training["employment_status"]=="Employed")
    if conditions.any():
        training_data.loc[i, 'employment_occupation'] = 'other_occupation'

In [6]:
for i, row in training[training['employment_industry'].isna()].iterrows():
    conditions = (training["employment_status"]=="Unemployed")
    if conditions.any():
        training.loc[i, 'employment_industry'] = 'unemployed_in_industry'

In [7]:
for i, row in training[training['employment_industry'].isna()].iterrows():
    conditions = (training["employment_status"]=="Not in Labor Force")
    if conditions.any():
        training.loc[i, 'employment_industry'] = 'Not_in_laborforce'

In [8]:
for i, row in training[training['employment_industry'].isna()].iterrows():
    conditions = (training["employment_status"]=="Employed")
    if conditions.any():
        training.loc[i, 'employment_industry'] = 'other_industry'

In [9]:
income_poverty_nan = 'refused_or_unknown'
marital_status_nan = 'refused_or_other_category'
employment_status_nan = 'refused_or_other_category'
rent_or_own_nan = 'other_or_refused'
education_nan = 'refused_or_unknown'


training['income_poverty'].fillna(income_poverty_nan, inplace=True)
training['marital_status'].fillna(marital_status_nan, inplace=True)
training['employment_status'].fillna(employment_status_nan, inplace=True)
training['rent_or_own'].fillna(rent_or_own_nan, inplace=True)
training['education'].fillna(education_nan, inplace=True)

In [10]:
numeric_columns = training.select_dtypes('number').columns

category_columns = ['race', 'sex', 'marital_status', 'rent_or_own',  'hhs_geo_region',
       'census_msa', 'employment_industry', 'employment_occupation']

ordinal_columns = ['age_group', 'education', 'income_poverty', 'employment_status']

In [11]:
training[category_columns].isna().sum()

race                     0
sex                      0
marital_status           0
rent_or_own              0
hhs_geo_region           0
census_msa               0
employment_industry      0
employment_occupation    0
dtype: int64

In [12]:
le = LabelEncoder()

In [13]:
for features in ordinal_columns:
    training[features] = le.fit_transform(training[features])

In [14]:
training[ordinal_columns].head()

,age_group,education,income_poverty,employment_status
0,3,1,2,1
1,1,0,2,0
2,0,2,0,0
3,4,0,2,1
4,2,3,0,0


In [15]:
training = pd.get_dummies(training, columns=category_columns, drop_first=True)

In [16]:
training_mode_imputation = training.copy()

In [17]:
training_mode_imputation.fillna(training_mode_imputation.mode().iloc[0], inplace=True)

In [18]:
training_mode_imputation.isna().sum(axis=0)

respondent_id                     0
h1n1_concern                      0
h1n1_knowledge                    0
behavioral_antiviral_meds         0
behavioral_avoidance              0
                                 ..
employment_occupation_vlluhbov    0
employment_occupation_xgwztkwe    0
employment_occupation_xqwwgdyp    0
employment_occupation_xtkaffoo    0
employment_occupation_xzmlyyjv    0
Length: 91, dtype: int64

In [19]:
labels = pd.read_csv('training_set_labels.csv')  # Training set features 

In [20]:
df_mode = pd.merge(training_mode_imputation, labels, on='respondent_id')

In [21]:
df_mode = df_mode.drop(['respondent_id'],axis=1)

In [22]:
X = df_mode.drop(['h1n1_vaccine','seasonal_vaccine'],axis=1)

In [23]:
y1 = df_mode[['h1n1_vaccine']]

In [24]:
y2 = df_mode[['seasonal_vaccine']]

In [25]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(X, y1, test_size=0.2, random_state=101)

In [26]:
scaler = StandardScaler()

In [27]:
scaled_X_train1 = scaler.fit_transform(X_train1)

scaled_X_test1 = scaler.transform(X_test1)

In [28]:
LR_model_h1n1 = LogisticRegression()

In [29]:
LR_model_h1n1.fit(scaled_X_train1, y_train1.values.ravel())

LogisticRegression()

In [30]:
LR_pred_h1n1 = LR_model_h1n1.predict(scaled_X_test1)

In [31]:
cm_h1n1 = confusion_matrix(y_test1, LR_pred_h1n1)
print(cm_h1n1)

[[3998  201]
 [ 686  457]]


In [32]:
print(classification_report(y_test1, LR_pred_h1n1))

              precision    recall  f1-score   support

           0       0.85      0.95      0.90      4199
           1       0.69      0.40      0.51      1143

    accuracy                           0.83      5342
   macro avg       0.77      0.68      0.70      5342
weighted avg       0.82      0.83      0.82      5342



In [33]:
from sklearn.metrics import accuracy_score

LR_acuuracy_h1n1 = accuracy_score(y_test1, LR_pred_h1n1)

LR_acuuracy_h1n1

0.8339573193560464

In [34]:
from sklearn.metrics import roc_auc_score

pred_proba_h1n1 = LR_model_h1n1.predict_proba(scaled_X_test1)[:, 1]

auc_roc_h1n1 = roc_auc_score(y_test1, pred_proba_h1n1)

print('AUC-ROC score for LR model H1n1 vaccine:', auc_roc_h1n1)

AUC-ROC score for LR model H1n1 vaccine: 0.8297990793541853


In [35]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y2, test_size=0.2, random_state=101)

In [36]:
scaler = StandardScaler()

scaled_X_train2 = scaler.fit_transform(X_train2)
scaled_X_test2 = scaler.transform(X_test2)

In [37]:
LR_model_seasonal = LogisticRegression()

In [38]:
LR_model_seasonal.fit(scaled_X_train2, y_train2.values.ravel())

LogisticRegression()

In [39]:
LR_pred_seasonal = LR_model_seasonal.predict(scaled_X_test2)

In [40]:
cm2 = confusion_matrix(y_test2, LR_pred_seasonal)
print(cm2)

[[2302  521]
 [ 674 1845]]


In [41]:
print(classification_report(y_test2, LR_pred_seasonal))

              precision    recall  f1-score   support

           0       0.77      0.82      0.79      2823
           1       0.78      0.73      0.76      2519

    accuracy                           0.78      5342
   macro avg       0.78      0.77      0.77      5342
weighted avg       0.78      0.78      0.78      5342



In [42]:
from sklearn.metrics import accuracy_score

LR_acuuracy_seasonal = accuracy_score(y_test2, LR_pred_seasonal)

LR_acuuracy_seasonal

0.7763010108573568

In [43]:
from sklearn.metrics import roc_auc_score

pred_proba_seasonal = LR_model_seasonal.predict_proba(scaled_X_test2)[:, 1] 
auc_roc_seasonal = roc_auc_score(y_test2, pred_proba_seasonal)

print("AUC-ROC score for LR model seasonal vaccine:", auc_roc_seasonal)

AUC-ROC score for LR model seasonal vaccine: 0.8535926111394


In [44]:


from sklearn.model_selection import RandomizedSearchCV

from scipy.stats import uniform

solver_combinations = {
    'liblinear': {'penalty': ['l1', 'l2'],
                  'C': [0.01, 0.1, 1, 10, 100]},
    'sag':       {'penalty': ['l2', 'none'],
                  'C': [0.01, 0.1, 1, 10, 100]},
    'saga':      {'penalty': ['l1', 'l2', 'elasticnet', 'none'],
                  'C': [0.001, 0.01, 0.1, 1, 10, 100]},
}

best_score = -1
best_params = None
LR_model_h1n1_tuned = None

for solver, combination in solver_combinations.items():
    
    param_distributions = {
        **combination,
        'solver': [solver],
        'fit_intercept': [True, False],
        'tol': [1e-4, 1e-5, 1e-6],
    }
    
    if solver == 'saga' and 'elasticnet' in combination['penalty']:
        param_distributions['l1_ratio'] = [0.1, 0.3, 0.5, 0.7, 0.9]

    random_search = RandomizedSearchCV(LR_model_h1n1, param_distributions, cv=5, n_jobs=-1, scoring='roc_auc', verbose=1, n_iter=100, random_state=42)
    random_search.fit(scaled_X_train1, y_train1.values.ravel())
    
    if random_search.best_score_ > best_score:
        best_score = random_search.best_score_
        best_params = random_search.best_params_
        LR_model_h1n1_tuned = random_search.best_estimator_

# Print the best hyperparameters
print("Best hyperparameters:", best_params)



Fitting 5 folds for each of 60 candidates, totalling 300 fits


C:\Users\SAYANTANI RAUTH\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:305: UserWarning: The total space of parameters 60 is smaller than n_iter=100. Running 60 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
C:\Users\SAYANTANI RAUTH\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:305: UserWarning: The total space of parameters 60 is smaller than n_iter=100. Running 60 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 100 candidates, totalling 500 fits


C:\Users\SAYANTANI RAUTH\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(


Best hyperparameters: {'tol': 1e-05, 'solver': 'liblinear', 'penalty': 'l1', 'fit_intercept': True, 'C': 0.1}


In [45]:
LR_model_h1n1_tuned.fit(scaled_X_train1, y_train1.values.ravel())

LogisticRegression(C=0.1, penalty='l1', solver='liblinear', tol=1e-05)

In [46]:
LR_pred_h1n1_tuned = LR_model_h1n1_tuned.predict(scaled_X_test1)

In [47]:
from sklearn.metrics import accuracy_score

LR_acuuracy_h1n1_tuned = accuracy_score(y_test1, LR_pred_h1n1_tuned)

LR_acuuracy_h1n1_tuned 


0.8333957319356047

In [48]:
from sklearn.metrics import roc_auc_score

pred_proba_h1n1_tuned = LR_model_h1n1_tuned.predict_proba(scaled_X_test1)[:, 1]

auc_roc_h1n1 = roc_auc_score(y_test1, pred_proba_h1n1_tuned)

print('AUC-ROC score for tuned LR model H1n1 vaccine:', auc_roc_h1n1)

AUC-ROC score for tuned LR model H1n1 vaccine: 0.8300418151470053


In [49]:


from sklearn.model_selection import RandomizedSearchCV

from scipy.stats import uniform

solver_combinations = {
    'liblinear': {'penalty': ['l1', 'l2'],
                  'C': [0.01, 0.1, 1, 10, 100]},
    'sag':       {'penalty': ['l2', 'none'],
                  'C': [0.01, 0.1, 1, 10, 100]},
    'saga':      {'penalty': ['l1', 'l2', 'elasticnet', 'none'],
                  'C': [0.01, 0.1, 1, 10, 100]},
}

best_score = -1
best_params = None
LR_model_seasonal_tuned = None

for solver, combination in solver_combinations.items():
    
    param_distributions = {
        **combination,
        'solver': [solver],
        'fit_intercept': [True, False],
        'tol': [1e-4, 1e-5, 1e-6],
    }
    
    if solver == 'saga' and 'elasticnet' in combination['penalty']:
        param_distributions['l1_ratio'] = [0.1, 0.3, 0.5, 0.7, 0.9]

    random_search = RandomizedSearchCV(LR_model_seasonal, param_distributions, cv=5, n_jobs=-1, scoring='roc_auc', verbose=1, n_iter=100, random_state=42)
    random_search.fit(scaled_X_train2, y_train2.values.ravel())
    
    if random_search.best_score_ > best_score:
        best_score = random_search.best_score_
        best_params = random_search.best_params_
        LR_model_seasonal_tuned = random_search.best_estimator_

# Print the best hyperparameters
print("Best hyperparameters:", best_params)



Fitting 5 folds for each of 60 candidates, totalling 300 fits


C:\Users\SAYANTANI RAUTH\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:305: UserWarning: The total space of parameters 60 is smaller than n_iter=100. Running 60 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
C:\Users\SAYANTANI RAUTH\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:305: UserWarning: The total space of parameters 60 is smaller than n_iter=100. Running 60 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 100 candidates, totalling 500 fits


C:\Users\SAYANTANI RAUTH\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(


Best hyperparameters: {'tol': 0.0001, 'solver': 'saga', 'penalty': 'l1', 'l1_ratio': 0.3, 'fit_intercept': True, 'C': 0.1}


In [50]:
LR_pred_seasonal_tuned = LR_model_seasonal_tuned.predict(scaled_X_test2)

In [51]:
from sklearn.metrics import accuracy_score

LR_acuuracy_seasonal_tuned = accuracy_score(y_test2, LR_pred_seasonal)

LR_acuuracy_seasonal_tuned

0.7763010108573568

In [52]:
from sklearn.metrics import roc_auc_score

pred_proba_seasonal_tuned = LR_model_seasonal_tuned.predict_proba(scaled_X_test2)[:, 1] 
auc_roc_seasonal_tuned = roc_auc_score(y_test2, pred_proba_seasonal_tuned)

print("AUC-ROC score for tuned LR model seasonal vaccine:", auc_roc_seasonal)

AUC-ROC score for tuned LR model seasonal vaccine: 0.8535926111394


In [53]:
import joblib


In [54]:
joblib.dump(LR_model_h1n1, 'LR_model_h1n1_mode.pkl')
joblib.dump(LR_model_seasonal, 'LR_model_seasonal_mode.pkl')
joblib.dump(LR_model_h1n1_tuned, 'LR_model_h1n1_tuned_mode.pkl')
joblib.dump(LR_model_seasonal_tuned, 'LR_model_seasonal_tuned_mode.pkl')


['LR_model_seasonal_tuned_mode.pkl']